In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00


In [3]:
!pip install torchmetrics==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 6.8 MB/s eta 0:00:00


In [4]:
!pip install pytorch-lightning
!pip install transformers
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00


In [5]:
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=d75c2a4c287fde7cea6f7d5f5f189d11a299cb0732b8483d2fdfbadde9c0cef8
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [6]:
!pip install timm==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 7.8 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import py
from sklearn.metrics import classification_report
from monai.data import decollate_batch, DataLoader, CacheDataset, ThreadDataLoader
from monai.metrics import ROCAUCMetric
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    ScaleIntensityd,
    EnsureTyped,
    Resized,
    CropForegroundd,
    SpatialPadd,
    CastToTyped,
    ConcatItemsd,
)
from monai.utils import set_determinism
import argparse
import math

In [8]:
import sys
import os
import requests

import torch
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.4.5  # 0.3.2 does not work in Colab
    !git clone https://github.com/facebookresearch/mae.git
    sys.path.append('./mae')
else:
    sys.path.append('..')
import models_mae

Running in Colab.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 0.4.12
    Uninstalling timm-0.4.12:
      Successfully uninstalled timm-0.4.12
Cloning into 'mae'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Receiving objects: 100% (39/39), 829.54 KiB | 19.29 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [9]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def show_image(image, title=''):
    # image is [H, W, 3]
    assert image.shape[2] == 3
    plt.imshow(torch.clip((image * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

def prepare_model(chkpt_dir, arch='mae_vit_large_patch16'):
    # build model
    model = getattr(models_mae, arch)()
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

def run_one_image(img, model):
    x = torch.tensor(img)

    # make it a batch-like
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)

    # run MAE
    loss, y, mask = model(x.float(), mask_ratio=0.75)
    y = model.unpatchify(y)
    y = torch.einsum('nchw->nhwc', y).detach().cpu()

    # visualize the mask
    mask = mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.patch_embed.patch_size[0]**2 *3)  # (N, H*W, p*p*3)
    mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()

    x = torch.einsum('nchw->nhwc', x)

    # masked image
    im_masked = x * (1 - mask)

    # MAE reconstruction pasted with visible patches
    im_paste = x * (1 - mask) + y * mask

    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 24]

    plt.subplot(1, 4, 1)
    show_image(x[0], "original")

    plt.subplot(1, 4, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 4, 3)
    show_image(y[0], "reconstruction")

    plt.subplot(1, 4, 4)
    show_image(im_paste[0], "reconstruction + visible")

    plt.show()

In [10]:
def set_seed(no):
    torch.manual_seed(no)
    random.seed(no)
    np.random.seed(no)
    os.environ['PYTHONHASHSEED'] = str()
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(100)

In [11]:
import pandas as pd
import os
from enum import Enum
from monai.transforms import Transform

from numpy.core.arrayprint import printoptions


class Modality(Enum):
    MR_T1 = 1
    MR_T2 = 2
    MR_T2_STAR = 3
    MR_FLAIR = 4
    MR_TOF_MRA = 5

def create_oasis_3_unimodal_dataset(csv_path, dataset_root, modality, transform,
                                    cache_rate):
    train_df = pd.read_csv(csv_path, sep=";")
    train_df.fillna('', inplace=True)

    column_name = ""
    if modality == Modality.MR_T1:
        column_name = "T1"
    elif modality == Modality.MR_T2:
        column_name = "T2"
    elif modality == Modality.MR_T2_STAR:
        column_name = "T2*"
    elif modality == Modality.MR_FLAIR:
        column_name = "FLAIR"
    elif modality == Modality.MR_TOF_MRA:
        column_name = "MRA"
    else:
        assert (False)

    train_data = []
    for index, row in train_df.iterrows():
        rel_path = row[column_name]
        if not rel_path:
            continue
        image_path = os.path.join(dataset_root, rel_path)
        train_data.append({"image": image_path, "label": row["label"]})

    # for root, dirs, files in os.walk(dataset_root):
    #     for file_name in files:
    #         file_path = os.path.join(root, file_name)
    #         train_data.append({"image": file_path, "label":0})

    return CacheDataset(data=train_data, transform=transform, cache_rate=cache_rate, num_workers=5,
                        copy_cache=False)
# modality_names = ["T1", "T2", "T2*", "FLAIR", "MRA"]

# def create_oasis_3_multimodal_dataset(csv_path: str, dataset_root: str, transform: Transform, cache_rate: float, missing_modality: str):
#     train_df = pd.read_csv(csv_path, sep=";")
#     train_df.fillna('', inplace=True)

#     train_data = []
#     for index, row in train_df.iterrows():
#         data_dict = {}
#         has_non_empty = False
#         for modality in modality_names:
#             file_path = row[modality]
#             if file_path:
#                 has_non_empty = True
#                 data_dict[modality] = os.path.join(dataset_root, file_path)
#             else:
#                 if missing_modality == "zeros":
#                     data_dict[modality] = "/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/empty_volume_2d.nii.gz"
#                 elif missing_modality == "gauss":
#                     data_dict[modality] = "/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz"
#                 else:
#                     raise ValueError(f"Invalid missing modality key {missing_modality}")
#         if not has_non_empty:
#             continue
#         data_dict["label"] = row["label"]
#         train_data.append(data_dict)
#     print(train_data)
#     return CacheDataset(data=train_data, transform=transform, cache_rate=cache_rate, num_workers=5, copy_cache=False)

# class SafeCropForegroundd:
#     def __init__(self, keys, source_key, select_fn, margin=0):
#         self.source_key = source_key
#         self.crop_foreground = CropForegroundd(keys=keys, source_key=source_key, select_fn=select_fn, margin=margin)

#     def __call__(self, data):
#         cropped_data = self.crop_foreground(data.copy())
#         cropped_image = cropped_data[self.source_key]

#         # Check if any dimension (excluding batch and channel dimensions) is zero.
#         if np.any(np.asarray(cropped_image.shape[1:]) == 0):
#             return data  # Revert to original data if cropped size is zero in any dimension

#         return cropped_data

In [12]:
!pip freeze > requirements.txt

In [13]:
dataset_root = r"/content/drive/MyDrive/oasis-3-2d-proc"

In [14]:
resolution = 224
cache_rate = 0.1 # might need to change this based on the amount of memory available
batch_size = 32
modality = Modality.MR_TOF_MRA

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
foreground_crop_threshold = 0.1
# train_table_path = "csv/oasis/oasis_3_multimodal_train.csv"
train_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_train_proc.csv"
transform = Compose([
    LoadImaged("image", image_only=True),
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    CastToTyped("label", dtype=np.float64),
    ScaleIntensityd("image"),
    CropForegroundd("image", source_key="image", select_fn=lambda x: x > foreground_crop_threshold, margin=5),
    Resized(keys=["image"], spatial_size=resolution, size_mode="longest"),
    SpatialPadd(keys=["image"], spatial_size=(resolution, resolution)),
    EnsureTyped("image", device=device),
    ]
)

# TODO: fix the following files
# TODO: skip them in the dataset
'''
torch.Size([96, 96, 26])
Error transforming file: /mnt/f/OASIS-3-MR-Sessions-2D/OAS30033_MR_d0133/anat5/NIFTI/sub-OAS30033_ses-d0133_run-02_T2w.nii.gz

torch.Size([64, 64, 105])
Error transforming file: /mnt/f/OASIS-3-MR-Sessions-2D/OAS31065_MR_d0044/anat7/NIFTI/sub-OAS31065_ses-d0044_echo-1_run-02_FLAIR.nii.gz
'''

train_ds = create_oasis_3_unimodal_dataset(csv_path=train_table_path, dataset_root=dataset_root, modality=modality, transform=transform, cache_rate=cache_rate)
train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=batch_size, shuffle=True)

# val_table_path = "csv/oasis/oasis_3_multimodal_val.csv"
val_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_val_proc.csv"
val_ds = create_oasis_3_unimodal_dataset(csv_path=val_table_path, dataset_root=dataset_root, modality=modality, transform=transform, cache_rate=cache_rate)
val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=batch_size, shuffle=True)

Loading dataset: 100%|██████████| 7/7 [00:02<00:00,  2.99it/s]


In [15]:
resolution = 224
cache_rate = 0.1
batch_size = 16
dataset_root = r"/content/drive/MyDrive/oasis-3-2d-proc"
missing_modality = "zeros"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
foreground_crop_threshold = 0.1
transform_list = [
        LoadImaged(keys=modality_names, image_only=True),
        EnsureChannelFirstd(keys=modality_names + ["label"], channel_dim="no_channel"),
        CastToTyped("label", dtype=np.float64),
        ScaleIntensityd(keys=modality_names),
    ]
for i in range(len(modality_names)):
     transform_list.append(
         SafeCropForegroundd(keys=modality_names[i], source_key=modality_names[i], select_fn=lambda x: x > foreground_crop_threshold, margin=5)
     )
transform_list.extend([
    Resized(keys=modality_names, spatial_size=resolution, size_mode="longest"),
    SpatialPadd(keys=modality_names, spatial_size=(resolution, resolution)),
    ConcatItemsd(keys=modality_names, name="image"),
    EnsureTyped(keys=["image"], device=device),
]
)
transform = Compose(transform_list)

train_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_train_proc.csv"
train_ds = create_oasis_3_multimodal_dataset(csv_path=train_table_path, dataset_root=dataset_root, transform=transform, cache_rate=cache_rate, missing_modality=missing_modality)
train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=batch_size, shuffle=True)

val_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_val_proc.csv"
val_ds = create_oasis_3_multimodal_dataset(csv_path=val_table_path, dataset_root=dataset_root, transform=transform, cache_rate=cache_rate, missing_modality=missing_modality)
val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=batch_size, shuffle=True)



NameError: ignored

In [16]:
# This is an MAE model trained with pixels as targets for visualization (ViT-Large, training mask ratio=0.75)

# download checkpoint if not exist
!wget -nc https://dl.fbaipublicfiles.com/mae/visualize/mae_visualize_vit_base.pth

chkpt_dir = 'mae_visualize_vit_base.pth'
mae = prepare_model(chkpt_dir, 'mae_vit_base_patch16')
print('Model loaded.')
print(mae(torch.ones([3,3,224,224])))

--2023-09-21 07:57:19--  https://dl.fbaipublicfiles.com/mae/visualize/mae_visualize_vit_base.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.251.15, 52.84.251.114, 52.84.251.27, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.251.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447725789 (427M) [binary/octet-stream]
Saving to: ‘mae_visualize_vit_base.pth’

mae_visualize_vit_b 100%[===================>] 426.98M  16.9MB/s    in 24s     

2023-09-21 07:57:44 (18.1 MB/s) - ‘mae_visualize_vit_base.pth’ saved [447725789/447725789]

<All keys matched successfully>
Model loaded.
(tensor(2.9613e-05, grad_fn=<DivBackward0>), tensor([[[0.6598, 0.6760, 0.6741,  ..., 0.6733, 0.6513, 0.7137],
         [0.9913, 0.9910, 0.9956,  ..., 1.0022, 1.0017, 1.0078],
         [0.9981, 0.9968, 0.9972,  ..., 0.9979, 1.0017, 1.0024],
         ...,
         [0.6780, 0.6784, 0.6808,  ..., 0.6664, 0.6796, 0.7322],
         [0.9991, 0.9972, 1.

In [17]:
class MAE(torch.nn.Module):
    def __init__(self, mae, num_classes=1) -> None:
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(3)
        )
        self.mae=mae




    def forward(self, img,mask_ratio=0.75):
        img = self.conv(img)
        latent, mask, ids_restore = self.mae.forward_encoder(img, mask_ratio)
        pred = self.mae.forward_decoder(latent, ids_restore)
          # [N, L, p*p*3]
        loss = self.mae.forward_loss(img, pred, mask)
        return loss, pred, mask
model = MAE(mae.cuda()).cuda()

In [19]:
optim = torch.optim.AdamW(model.parameters(), lr=0.003 * 12 / 256, betas=(0.9, 0.95), weight_decay=0.05)
lr_func = lambda epoch: min((epoch + 1) / (10 + 1e-8), 0.5 * (math.cos(epoch / 150 * math.pi) + 1))
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)

step_count = 0
optim.zero_grad()
from tqdm import tqdm
for e in range(150):
  model.train()
  losses = []

  for  batch_data in tqdm(train_loader):
      img, label= batch_data["image"].to(device), batch_data["label"].to(device)
      # img = batch_data["image"].to(device)
      step_count += 1
      # img = img.to(device)
      # predicted_img, mask = model(img)

      loss, _, _ = model(img, mask_ratio=0.75)

      # loss = torch.mean((predicted_img - img) ** 2 * mask) / 0.75
      loss.backward()
      if step_count % 1 == 0:
          optim.step()
          optim.zero_grad()
      losses.append(loss.item())
  lr_scheduler.step()
  avg_loss = sum(losses) / len(losses)
  # writer.add_scalar('mae_loss', avg_loss, global_step=e)
  print(f'In epoch {e}, average traning loss is {avg_loss}.')

  ''' visualize the first 16 predicted images on val dataset'''
  model.eval()
  torch.save(model,'/content/drive/MyDrive/pretrained_with_preoasismra.pt')

Adjusting learning rate of group 0 to 1.4062e-05.


100%|██████████| 22/22 [12:07<00:00, 33.05s/it]


Adjusting learning rate of group 0 to 2.8125e-05.
In epoch 0, average traning loss is 0.02108584437519312.


100%|██████████| 22/22 [00:13<00:00,  1.65it/s]


Adjusting learning rate of group 0 to 4.2187e-05.
In epoch 1, average traning loss is 0.018686387518590145.


...

Adjusting learning rate of group 0 to 6.1676e-08.
In epoch 147, average traning loss is 0.005147579401223497.


100%|██████████| 22/22 [00:13<00:00,  1.66it/s]


Adjusting learning rate of group 0 to 1.5421e-08.
In epoch 148, average traning loss is 0.0051566696387122975.


100%|██████████| 22/22 [00:13<00:00,  1.67it/s]


Adjusting learning rate of group 0 to 0.0000e+00.
In epoch 149, average traning loss is 0.005207129097966986.
